In [1]:
# Mount into drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# Free colab seems to give different amount of general RAM to different users or even the same users at different times.
!free -h

# check which nvidia drivers and cuda version is running
!nvidia-smi
!nvidia-smi -L

               total        used        free      shared  buff/cache   available
Mem:            83Gi       890Mi        78Gi       1.0Mi       4.1Gi        81Gi
Swap:             0B          0B          0B
Tue Apr 23 02:24:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              44W / 400W

In [3]:
# declare env vars
%env PROJECT_DIR=/content/drive/My Drive/Education/Master's (GaTech)/Courses/CS7643: Deep Learning/Project/CS7643-Efficient_LLM
%env model_name=facebook/opt-125m

env: PROJECT_DIR=/content/drive/My Drive/Education/Master's (GaTech)/Courses/CS7643: Deep Learning/Project/CS7643-Efficient_LLM
env: model_name=facebook/opt-125m


In [4]:

# need to match the system-wide installed cuda-11 for deepspeed to compile
# so install the matching pytorch

# pt-1.8.1 works too
# !pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

# pt-1.11
#!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install -U torch==2.2.1+cu121 torchvision==0.17.1+cu121 torchaudio==2.2.1+cu121 -f https://download.pytorch.org/whl/cu121/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu121/torch_stable.html
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_

In [5]:
# either install the release
#!pip install deepspeed
# or the master

!pip install -U git+https://github.com/microsoft/deepspeed
# !pip install -U git+https://github.com/microsoft/DeepSpeed/tree/v0.8.3


# remove any previously cached deepspeed objects as they can be incompatible with this new build
#!rm -r /root/.cache/torch_extensions/

  Cloning https://github.com/microsoft/deepspeed to /tmp/pip-req-build-jkauw_i6
  Running command git clone --filter=blob:none --quiet https://github.com/microsoft/deepspeed /tmp/pip-req-build-jkauw_i6
  Resolved https://github.com/microsoft/deepspeed to commit c66bc4269e9484b6e57d6f5521df02c70d399246
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.2 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.14.2+c66bc426-py3-none-any.whl size=1431560 sha256=731c70e579f4853009901b9d6cd6257216ddd0f57f3644031bf82d51e0745422
  Stored in directory: /tmp/pip-ephem-wheel-cache-f659gas0/wheels/c7/2a/60/6cd1be5e9093337b6243efab085ade35b787b5c4c1aaf99387
Successfully built deepspeed


In [6]:
!pip install -U datasets evaluate==0.3.0 transformers==4.30.0 accelerate==0.27.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 45.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20

In [7]:
import torch
print(torch.cuda.device_count())
print(torch.cuda.is_available())

1
True


In [ ]:
# ICL experiment
# bash $PROJECT_DIR/scripts/in_context/mnli/run_gpt3.sh mnli 16 facebook/opt-30b 1 60000
# Train custom model
#!python eval.py --task_name mnli --num_shots 2 --model_name_or_path facebook/opt-125m --pattern "{text1} {text2} ?" --target_prefix " " --target_tokens "ĠYes,ĠNo" --separate_shots_by "\n\n" --group "minimal" --max_seq_length 2048 --output_dir "./output" --do_eval --eval_task_name "hans" --per_device_eval_batch_size 10 --balanced --shuffle --seed 0 --data_seed 0 --dataset_cache_dir "./hf_dataset" --cache_dir "./hf_model" --report_to "none"
!python "${PROJECT_DIR}/eval.py" \
        --model_name_or_path $model_name \
        --cache_dir "./hf_model" \
        --task_name mnli \
        --pattern "{text1} question: {text2} Yes or No?" \
        --target_prefix " answer: " \
        --target_tokens "ĠYes,ĠNo" \
        --separate_shots_by "\n\n" \
        --group "gpt-3" \
        --dataset_cache_dir "./hf_dataset" \
        --max_seq_length 2048 \
        --output_dir "./output" \
        --do_eval  \
        --eval_task_name "hans" \
        --per_device_eval_batch_size 10 \
        --num_shots 16 \
        --balanced \
        --shuffle \
        --fp16 \
        --seed 0 \
        --data_seed 0 \
        --report_to "none"


2024-04-14 07:44:04.640439: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 07:44:04.640496: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 07:44:04.641949: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-14 07:44:05.845600: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2024-04-14 07:44:12,105] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: pleas

In [8]:
# FT experiment (with LM model)
# args: task_name, max_train_samples, epochs, warmup_ratio, bsz, num_gpus, learning_rate, model_name_or_path, port
#bash $PROJECT_DIR/scripts/pattern_verbalizer_ft/mnli/run.sh mnli 128 40 0.5 4 8 1e-5 facebook/opt-13b 60000


!python "${PROJECT_DIR}/ft.py" \
            --wandb_project_name llmft-experiments \
            --wandb_group_name vanilla-ft \
            --model_name_or_path $model_name \
            --cache_dir "./hf_model" \
            --task_name mnli \
            --pattern "{text1} {text2} ?" \
            --target_tokens "ĠYes,ĠNo" \
            --dataset_cache_dir "./hf_dataset" \
            --max_seq_length 256 \
            --output_dir "./output" \
            --overwrite_output_dir \
            --do_train \
            --max_train_samples 128 \
            --per_device_train_batch_size 4 \
            --gradient_accumulation_steps 1 \
            --num_train_epochs 40 \
            --warmup_ratio 0.5 \
            --logging_first_step false \
            --logging_steps -1 \
            --learning_rate 1e-5 \
            --weight_decay 0.0 \
            --do_eval \
            --evaluation_strategy epoch \
            --per_device_eval_batch_size 10 \
            --eval_on_hans \
            --save_strategy no \
            --fp16 \
            --seed 0 \
            --data_seed 0 \
            --report_to "none" \
            2> "${PROJECT_DIR}/logfiles/console.err"

[2024-04-22 16:56:43,317] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible
Sample 98 of the training set: {'premise': "you know Uncle Sam but i've always thought that they were slow  as far as outside of the government you know", 'hypothesis': "My personal opinion is that they've always been fast."

In [11]:
# FT with Context Distillation experiment
# args: task_name, max_train_samples, epochs, warmup_ratio, bsz, num_gpus, learning_rate, model_name_or_path, port
#bash $PROJECT_DIR/scripts/pattern_verbalizer_ft/mnli/run.sh mnli 128 40 0.5 4 8 1e-5 facebook/opt-13b 60000

# --pattern "{text1} question: {text2} Yes or No?" \
# --pattern "{text1} {text2} ?" \
# --pattern "{text1} question: {text2} Yes or No? answer:" \


!python "${PROJECT_DIR}/ft.py" \
            --wandb_project_name llmft-experiments \
            --wandb_group_name pattern-verbalizer-ft-context-distillation \
            --model_name_or_path $model_name \
            --cache_dir "./hf_model" \
            --task_name mnli \
            --target_tokens "ĠYes,ĠNo" \
            --pattern "{text1} question: {text2} Yes or No?" \
            --dataset_cache_dir "./hf_dataset" \
            --max_seq_length 256 \
            --context_max_seq_length 2048 \
            --output_dir "./output" \
            --overwrite_output_dir \
            --do_train \
            --max_train_samples 128 \
            --per_device_train_batch_size 4 \
            --gradient_accumulation_steps 1 \
            --num_train_epochs 40 \
            --warmup_ratio 0.5 \
            --logging_first_step false \
            --logging_steps -1 \
            --learning_rate 1e-5 \
            --weight_decay 0.0 \
            --do_eval \
            --context_distillation_flag True \
            --evaluation_strategy epoch \
            --per_device_eval_batch_size 10 \
            --eval_on_hans \
            --save_strategy no \
            --fp16 \
            --seed 0 \
            --data_seed 0 \
            --num_shots 16 \
            --context_target_tokens "ĠYes,ĠNo" \
            --context_pattern "{text1} question: {text2} Yes or No?" \
            --separate_shots_by "\n\n" \
            --target_prefix " answer: " \
            --balanced  \
            --shuffle  \
            --report_to "none" \
            2> "${PROJECT_DIR}/logfiles/console_err.txt"

[2024-04-23 06:59:47,785] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible
Sample 98 of the training set: {'premise': "They're some way from the chateau and each other", 'hypothesis': 'They are not near to each other.', 'label': 3216, 'idx': 192625, 'input_ids': [2, 1213, 214, 103, 169, 31, 5, 1855, 877, 1180, 8, 349, 97, 864, 35, 252, 32, 45, 583, 7, 349, 97, 4, 3216, 50, 440,